In [18]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib import request, parse
import time

In [19]:
# Takes a country name, returns the table of cities and prices from that country's page on pintprice.com
def get_prices(country):
    safe = parse.quote(country)
    url = 'http://www.pintprice.com/region.php?/{}/USD.htm'.format(safe)
    html = request.urlopen(url)
    if html.status == 200:
        soup = BeautifulSoup(html.read(), 'html.parser')
        table = []
        tds = [td.get_text().strip() for td in soup.find_all("td")]
        for x in range(0, len(tds), 2):
            table.append([tds[x], tds[x+1]])
        return table
    else:
        return "Not Found"

In [20]:
# Parse one page to get the list of all countries from the drop-down
uk = parse.quote("United Kingdom")
url = 'http://www.pintprice.com/region.php?/{}/USD.htm'.format(uk)
html = request.urlopen(url)
soup = BeautifulSoup(html.read(), 'html.parser')
countries = [c.get_text() for c in soup.find_all("option")][:-5]

In [21]:
# There's an extra <td> on the UK page, so let's skip it
pint_price = {}
uk_table = []
uk_tds = [td.get_text().strip() for td in soup.find_all("td")][1:]
for x in range(0, len(uk_tds), 2):
    uk_table.append([uk_tds[x], uk_tds[x+1]])
pint_price["United Kingdom"] = uk_table

In [22]:
for c in countries:
    if c != "United Kingdom":
        pint_price[c] = get_prices(c)
        time.sleep(0.1)

In [31]:
# Move Cost/Price to the column headings
pint_price_df = pd.DataFrame(pint_price['United Kingdom'])
pint_price_df.columns = pint_price_df.iloc[0]
pint_price_df = pint_price_df.drop(pint_price_df.index[0])

# set the Country column and clean up the Price column
pint_price_df["Country"] = "United Kingdom"
pint_price_df['Price'] = pint_price_df['Price'].str.strip('$ USD').astype(float)

In [32]:
# Do it again for each country and concat onto pint_price_df
for country in pint_price:
    if country != "United Kingdom":
        pp = pd.DataFrame(pint_price[country])
        pp.columns = pp.iloc[0]
        pp = pp.drop(pp.index[0])
        
        pp['Country'] = country
        pp['Price'] = pp['Price'].str.strip("$ USD").astype(float, errors="ignore")
        
        pint_price_df = pd.concat([pint_price_df, pp])

In [33]:
pint_price_df.columns = ['city_ascii', 'beer_pub', 'country']
pint_price_df = pint_price_df.reindex(columns=['city_ascii', 'country', 'beer_pub'])

In [34]:
pint_price_df.to_csv("pintprice.csv")

In [35]:
pint_price_df

,city_ascii,country,beer_pub
1,,United Kingdom,4.17
2,Aberdeen,United Kingdom,5.07
3,Aberystwyth,United Kingdom,3.58
4,Acton,United Kingdom,6.26
5,Airdrie,United Kingdom,3.43
...,...,...,...
2,Lusaka,Zambia,1.5
3,Petauke,Zambia,0.91
4,Solwezi,Zambia,1.4
1,Bulawayo,Zimbabwe,1.13
